In [2]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import time

In [3]:
links = {}
driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")
driver.get("https://www.tripadvisor.com/Search?q=parks&searchSessionId=CF4A26D0C451E0783853C96C46AC8B131637532036921ssid&sid=72ED914F7F2B4263B4C7FB9FF61A5A671637532046511&geo=186338&blockRedirect=true&ssrc=a")
javaScript = "PagerNextClick();"

C:\Users\wirze\AppData\Local\Temp/ipykernel_26152/1593989845.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\wirze\\DataspellProjects\\chromedriver_win32\\chromedriver.exe")


In [4]:
inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
inputElement.send_keys('Moscow')

C:\Users\wirze\AppData\Local\Temp/ipykernel_26152/1130507203.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")


In [5]:
clearElement = driver.find_element_by_id("CLEAR_WHERE")
clearElement.click()

C:\Users\wirze\AppData\Local\Temp/ipykernel_26152/3341757118.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  clearElement = driver.find_element_by_id("CLEAR_WHERE")


In [6]:
inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")
inputElement.send_keys('London')

C:\Users\wirze\AppData\Local\Temp/ipykernel_26152/301636850.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver.find_element_by_id("GEO_SCOPED_SEARCH_INPUT")


In [7]:
inputElement.send_keys(Keys.ENTER)

In [8]:
parksElement = driver.find_elements_by_xpath("//*[contains(text(), 'Parks in')]")
parksElement[1].click()

C:\Users\wirze\AppData\Local\Temp/ipykernel_26152/1667884103.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  parksElement = driver.find_elements_by_xpath("//*[contains(text(), 'Parks in')]")


In [10]:
for i in range(42):
    html = driver.page_source
    soup = BeautifulSoup(html)

    results_names = soup.find_all("div", class_="csemS")
    results_links = soup.find_all("div", class_="fVbwn cdAAV cagLQ eZTON dofsx")
    del results_links[1::2]  # delete every second element (counting from the first)

    for result_link, result_name in zip(results_links, results_names):
        if result_link.find("a") is not None:
            links[result_name.text] = result_link.find("a")['href']

    NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')
    if len(NextPageElement) == 0:
        break
    NextPageElement[0].click()
    time.sleep(5)

C:\Users\wirze\AppData\Local\Temp/ipykernel_26152/2444517317.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  NextPageElement = driver.find_elements_by_css_selector('[aria-label="Next page"]')


In [11]:
links

{'1. Hyde Park': '/Attraction_Review-g186338-d189030-Reviews-Hyde_Park-London_England.html',
 "2. St. James's Park": '/Attraction_Review-g186338-d188126-Reviews-St_James_s_Park-London_England.html',
 "3. Regent's Park": '/Attraction_Review-g186338-d187675-Reviews-Regent_s_Park-London_England.html',
 '4. Kensington Gardens': '/Attraction_Review-g186338-d188893-Reviews-Kensington_Gardens-London_England.html',
 '5. Hampstead Heath': '/Attraction_Review-g186338-d211768-Reviews-Hampstead_Heath-London_England.html',
 '6. Holland Park': '/Attraction_Review-g186338-d187683-Reviews-Holland_Park-London_England.html',
 '7. Greenwich Park': '/Attraction_Review-g186338-d211600-Reviews-Greenwich_Park-London_England.html',
 '8. Primrose Hill': '/Attraction_Review-g186338-d2464189-Reviews-Primrose_Hill-London_England.html',
 '9. Battersea Park': '/Attraction_Review-g186338-d2494059-Reviews-Battersea_Park-London_England.html',
 '10. Green Park': '/Attraction_Review-g186338-d211766-Reviews-Green_Park-Lo

In [12]:
df_links = pd.DataFrame.from_dict(links,  orient='index', columns=["link"]).reset_index()
df_links.to_csv("london_parks_links.csv")
df_links

,index,link
0,1. Hyde Park,/Attraction_Review-g186338-d189030-Reviews-Hyd...
1,2. St. James's Park,/Attraction_Review-g186338-d188126-Reviews-St_...
2,3. Regent's Park,/Attraction_Review-g186338-d187675-Reviews-Reg...
3,4. Kensington Gardens,/Attraction_Review-g186338-d188893-Reviews-Ken...
4,5. Hampstead Heath,/Attraction_Review-g186338-d211768-Reviews-Ham...
...,...,...
146,162. Martin Luther King Adventure Playground,/Attraction_Review-g186338-d12097677-Reviews-M...
147,163. Timbuktu Adventure Playground,/Attraction_Review-g186338-d12097680-Reviews-T...
148,164. Cannizaro Park,/Attraction_Review-g186338-d13544032-Reviews-C...
149,165. Gladstone Park,/Attraction_Review-g186338-d16729335-Reviews-G...


In [13]:
driver.quit()

In [128]:
n_pages = 1500
for i in range(n_pages):
    try:
        driver.execute_script(javaScript)
        # driver.implicitly_wait(3) # seconds

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        results = soup.find_all("a", class_="appeal-title-link")
        for result in results:
            links[re.findall(r'\d+', result['href'])[0]] = result['href']

    finally:
        print(i/n_pages)

driver.quit()

0.0
0.0006666666666666666
0.0013333333333333333
0.002
0.0026666666666666666
0.0033333333333333335
0.004
0.004666666666666667
0.005333333333333333
0.006
0.006666666666666667
0.007333333333333333
0.008
0.008666666666666666
0.009333333333333334
0.01
0.010666666666666666
0.011333333333333334
0.012
0.012666666666666666
0.013333333333333334
0.014
0.014666666666666666
0.015333333333333332
0.016
0.016666666666666666
0.017333333333333333
0.018
0.018666666666666668
0.019333333333333334
0.02
0.020666666666666667
0.021333333333333333
0.022
0.02266666666666667
0.023333333333333334
0.024
0.024666666666666667
0.025333333333333333
0.026
0.02666666666666667
0.027333333333333334
0.028
0.028666666666666667
0.029333333333333333
0.03
0.030666666666666665
0.03133333333333333
0.032
0.03266666666666666
0.03333333333333333
0.034
0.034666666666666665
0.035333333333333335
0.036
0.03666666666666667
0.037333333333333336
0.038
0.03866666666666667
0.03933333333333333
0.04
0.04066666666666666
0.04133333333333333
0.04

In [130]:
links_messages = {}
links_headings = {}

In [131]:
for link in links:
    URL = "http://zpp.rospotrebnadzor.ru"+links[link]
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    page_message = soup.find("p", class_="appeal-details-message")
    page_heading = soup.find("h2")

    links_messages[link] = page_message.text
    links_headings[link] = page_heading.text

In [132]:
len(links_messages)

2150

In [133]:
df_links = pd.DataFrame.from_dict(links,  orient='index', columns=["link"]).reset_index()
df_links

,index,link
0,30056,/Forum/Appeals/Details/30056
1,30047,/Forum/Appeals/Details/30047
2,30043,/Forum/Appeals/Details/30043
3,30037,/Forum/Appeals/Details/30037
4,30016,/Forum/Appeals/Details/30016
...,...,...
2145,26658,/Forum/Appeals/Details/26658
2146,26654,/Forum/Appeals/Details/26654
2147,26652,/Forum/Appeals/Details/26652
2148,26651,/Forum/Appeals/Details/26651


In [134]:
df_messages = pd.DataFrame.from_dict(links_messages,  orient='index', columns=["message"]).reset_index()
df_messages

,index,message
0,30056,"Прошу предоставить правовую информацию, на осн..."
1,30047,Здравствуйте! По договору розничной купли-про...
2,30043,"здравствуйте, я Шагеева Ольга Владимировна, пр..."
3,30037,Здравствуйте! В сети интернет увидела введение...
4,30016,"Здравствуйте! Пожалуйста, примите меры ! https..."
...,...,...
2145,26658,"Как и куда отправить ответ ПЦР-теста, если нет..."
2146,26654,Здравствуйте! Сайт Zengram списали с моей карт...
2147,26652,Прошу помочь урегулировать вопрос по страховым...
2148,26651,"Добрый день! Сегодня обнаружила, что вчера веч..."


In [135]:
df_headings = pd.DataFrame.from_dict(links_headings,  orient='index', columns=["heading"]).reset_index()
df_headings

,index,heading
0,30056,Заявление-разьяснение
1,30047,Уклонение от исполнения договора
2,30043,торговля подделкой красной икры
3,30037,Вопрос о втором законопроекте
4,30016,Мошенники
...,...,...
2145,26658,Как отправить ПЦР тест?
2146,26654,Отказали в возврате денежных средств за неиспо...
2147,26652,Мошеннические схемы ГАЗЭНЕРГОБАНКа
2148,26651,Сайт отказал в возврате средств за неиспользов...


In [136]:
df = df_links.merge(df_headings).merge(df_messages)
df.to_csv("small_data.csv")
df

,index,link,heading,message
0,30056,/Forum/Appeals/Details/30056,Заявление-разьяснение,"Прошу предоставить правовую информацию, на осн..."
1,30047,/Forum/Appeals/Details/30047,Уклонение от исполнения договора,Здравствуйте! По договору розничной купли-про...
2,30043,/Forum/Appeals/Details/30043,торговля подделкой красной икры,"здравствуйте, я Шагеева Ольга Владимировна, пр..."
3,30037,/Forum/Appeals/Details/30037,Вопрос о втором законопроекте,Здравствуйте! В сети интернет увидела введение...
4,30016,/Forum/Appeals/Details/30016,Мошенники,"Здравствуйте! Пожалуйста, примите меры ! https..."
...,...,...,...,...
2145,26658,/Forum/Appeals/Details/26658,Как отправить ПЦР тест?,"Как и куда отправить ответ ПЦР-теста, если нет..."
2146,26654,/Forum/Appeals/Details/26654,Отказали в возврате денежных средств за неиспо...,Здравствуйте! Сайт Zengram списали с моей карт...
2147,26652,/Forum/Appeals/Details/26652,Мошеннические схемы ГАЗЭНЕРГОБАНКа,Прошу помочь урегулировать вопрос по страховым...
2148,26651,/Forum/Appeals/Details/26651,Сайт отказал в возврате средств за неиспользов...,"Добрый день! Сегодня обнаружила, что вчера веч..."
